In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2
import tensorflow as tf
from sklearn.metrics import jaccard_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [2]:
dirname_img = 'data_2/images/'
dirname_mask = 'data_2/masks/'
images = list()
masks = list()
for imagepath, maskpath in zip(os.listdir(dirname_img), os.listdir(dirname_mask) ):
    #print(dirname_img + imagepath)
    image = cv2.imread(dirname_img + imagepath)
    image = cv2.resize(image, (256, 256))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = np.reshape(image, (256,256,-1))
    images.append(image)
    
    mask = cv2.imread(dirname_img + maskpath)
    mask = cv2.resize(mask, (256, 256))
    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    mask = np.reshape(mask, (256,256,-1))
    masks.append(mask)
    
images = np.array(images)/255
masks = np.array(np.array(masks)/255 > 0.5, dtype='f')
    
print(images.shape)
print(masks.shape)

(366, 256, 256, 1)
(366, 256, 256, 1)


In [3]:
mat = np.random.choice(a=[False, True], size=(len(images)), p=[0.75, 0.25])
validation_images = images[mat]
validation_masks = masks[mat]

images = images[~mat]
masks = masks[~mat]

mat = np.random.choice(a=[False, True], size=(len(images)), p=[0.7, 0.3])

test_images = images[mat]
test_masks = masks[mat]

images = images[~mat]
masks = masks[~mat]

print(f'Train {len(images)}, Validation {len(validation_images)}, Test {len(test_images)}')

Train 194, Validation 82, Test 90


In [4]:
EPOCHS = 100
BATCH_SIZE = 8
FEATURE_CHANNELS = [32,64,128,256,512]
IMAGE_RES = 256
SEED = 111
AUGUMENTATION_PARAMETERS = {
    'rotation_range': 30,
    'zoom_range': 0.1,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'fill_mode': 'constant',
    'cval': 0.0,
    'shear_range': 0.2,
    'horizontal_flip':True,
    'vertical_flip': True
    
}

In [5]:
datagen = ImageDataGenerator(**AUGUMENTATION_PARAMETERS)
validation_datagen = ImageDataGenerator() 

image_iterator = datagen.flow(images, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)
mask_iterator = datagen.flow(masks, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)
train_iterator = zip(image_iterator, mask_iterator)
train_steps = len(image_iterator)

valid_image_iterator = validation_datagen.flow(validation_images, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)
valid_mask_iterator = validation_datagen.flow(validation_masks, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)
valid_iterator = zip(valid_image_iterator, valid_mask_iterator)
valid_steps = len(valid_image_iterator)

print(f'Train steps {train_steps}\nValidation steps {valid_steps}')

Train steps 25
Validation steps 11


In [6]:
def down_block(x, filters, kernel_size=(3,3), padding="same", strides=1):
    with tf.device('/device:GPU:0'):
        c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
        c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)

        p = tf.keras.layers.MaxPool2D((2,2), (2,2))(c)
    return c, p 

def up_block(x, skip, filters, kernel_size=(3,3), padding="same", strides=1):
    with tf.device('/device:GPU:0'):
        up_sampling = tf.keras.layers.UpSampling2D((2,2))(x)
        concat = tf.keras.layers.Concatenate()([up_sampling, skip])

        c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(concat)
        c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)

    return c

def bottleneck(x, filters, kernel_size=(3,3), padding="same", strides=1):
    with tf.device('/device:GPU:0'):
        c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
        c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)

    return c

def UNet(feature_channels, image_size, input_channels=1):
    feature_maps = feature_channels  #[32,64,128,256,512]
    inputs = tf.keras.layers.Input( (image_size, image_size, input_channels) )

    pool_0 = inputs
    conv_1, pool_1 = down_block(pool_0, feature_maps[0]) 
    conv_2, pool_2 = down_block(pool_1, feature_maps[1]) 
    conv_3, pool_3 = down_block(pool_2, feature_maps[2]) 
    conv_4, pool_4 = down_block(pool_3, feature_maps[3]) 

    bn = bottleneck(pool_4, feature_maps[4])

    ups_1 = up_block(bn, conv_4, feature_maps[3]) 
    ups_2 = up_block(ups_1, conv_3, feature_maps[2]) 
    ups_3 = up_block(ups_2, conv_2, feature_maps[1]) 
    ups_4 = up_block(ups_3, conv_1, feature_maps[0]) 

    outputs = tf.keras.layers.Conv2D(1, (1,1), padding='same', activation='sigmoid')(ups_4)

    model = tf.keras.models.Model(inputs, outputs)
    return model

In [7]:
model  = UNet(FEATURE_CHANNELS, IMAGE_RES, images.shape[-1])  
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=[
        'acc',
        tf.keras.metrics.Precision(),
        tf.keras.metrics.AUC(),
        tf.keras.metrics.Recall()
    ]
)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 32  9248        ['conv2d[0][0]']                 
                                )                                                             

Total params: 7,846,081
Trainable params: 7,846,081
Non-trainable params: 0
__________________________________________________________________________________________________
None


In [8]:
model.fit(
    train_iterator,
    steps_per_epoch=train_steps,
    epochs=10,
    validation_steps=train_steps,
    validation_data=train_iterator,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True, verbose=1) 
        ]

)

Epoch 1/10
25/25 [==============================] - 151s 6s/step - loss: 0.4447 - acc: 0.8857 - precision: 0.0149 - auc: 0.5275 - recall: 1.9791e-04 - val_loss: 0.1353 - val_acc: 0.8882 - val_precision: 0.0000e+00 - val_auc: 0.9726 - val_recall: 0.0000e+00
Epoch 2/10
25/25 [==============================] - 143s 6s/step - loss: 0.1067 - acc: 0.9031 - precision: 0.9934 - auc: 0.9870 - recall: 0.1252 - val_loss: 0.0855 - val_acc: 0.9624 - val_precision: 0.9917 - val_auc: 0.9819 - val_recall: 0.6932
Epoch 3/10
25/25 [==============================] - 142s 6s/step - loss: 0.0432 - acc: 0.9667 - precision: 0.9565 - auc: 0.9939 - recall: 0.8506 - val_loss: 0.0321 - val_acc: 0.9692 - val_precision: 0.9870 - val_auc: 0.9970 - val_recall: 0.8226
Epoch 4/10
25/25 [==============================] - 141s 6s/step - loss: 0.0298 - acc: 0.9698 - precision: 0.9805 - auc: 0.9972 - recall: 0.8723 - val_loss: 0.0265 - val_acc: 0.9711 - val_precision: 0.9896 - val_auc: 0.9982 - val_recall: 0.8722
Epoch 5/